In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.metrics import accuracy_score

In [3]:
import pickle

In [4]:
path = 'C:/Users/lyc07/Desktop/Columbia/S3/GR5243 Applied Data Science/prj3/'
dat_train = pd.read_csv(path + "dat_train.csv")
dat_test = pd.read_csv(path + 'dat_test.csv')

In [5]:
dat_full = pd.concat([dat_train, dat_test], ignore_index = True)
dat_full.head()

,Unnamed: 0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature5998,feature5999,feature6000,feature6001,feature6002,feature6003,feature6004,feature6005,feature6006,emotion_idx
0,1,31,18,1,20,33,20,1,17,151,...,98,150,202,50,102,154,52,104,52,21
1,2,35,15,6,27,39,20,2,17,151,...,104,159,213,54,109,163,55,109,54,11
2,3,35,17,2,20,32,14,3,20,165,...,99,152,204,52,105,157,53,105,52,2
3,4,33,16,3,20,29,12,6,23,161,...,101,156,210,54,109,163,55,109,54,7
4,5,29,15,1,17,27,13,2,17,133,...,83,128,172,44,89,133,45,89,44,1


In [7]:
#dat_full.to_csv(path + 'dat_full.csv')

In [6]:
dat_full.drop('Unnamed: 0', inplace=True, axis=1)

In [7]:
X, y = dat_full.iloc[:,:-1],dat_full.iloc[:,-1]

In [29]:
y = y-1

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

2386    17
1298     4
1995    10
2097     4
289     21
Name: emotion_idx, dtype: int64

In [31]:
# dataset
# feature_name = features
# , categorical_feature = categorical_features
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test)

In [32]:
# Training
# learning_rate
# max_depth
# num_leaves
# min_data_in_leaf
# parameter for categorical values: categorical_feature
# parameter for conrolling speed: feature_fraction; bagging_fraction; num_iterations
params = {
 'task': 'train'
 , 'boosting_type': 'gbdt'
 , 'objective': 'multiclass'
 , 'num_class': 22
 , 'metric':'multi_logloss'
 , 'min_data': 1
 , 'verbose': -1
}
 
gbm = lgb.train(params, lgb_train, num_boost_round=50)

In [33]:
# prediction
predictions = gbm.predict(X_test)

In [34]:
predictions.shape

(500, 22)

In [35]:
pred_index = np.argmax(predictions, axis = 1)

In [36]:
pred_index

array([16,  3,  9, 12,  1, 14, 14, 15, 19, 19, 18,  1, 13, 10,  8, 15, 18,
       13, 15,  9, 20, 15, 21,  5, 13,  7, 19,  8, 20, 21,  3, 15,  2,  5,
        3,  2, 15, 14,  3,  0, 10,  8,  4,  9, 12,  7,  2,  5,  2, 18,  2,
       13, 18,  8, 18, 17, 15, 14,  2,  2, 17, 20, 10,  0, 10, 15, 11, 16,
       19,  3,  3, 10,  6, 21, 13,  7,  2, 19, 19, 18, 14,  0, 18,  0, 13,
        5, 15, 20,  5,  2,  9, 18,  5,  4, 17,  0, 16, 19, 19,  2, 20,  9,
       13, 20,  7, 16, 18,  9, 14,  0,  0, 16, 20,  7, 21,  9,  3, 10,  3,
       20,  9,  8,  3, 16, 11, 17, 10, 11,  7, 13,  4, 11, 21, 11, 11,  2,
        6, 17, 15,  1, 16,  8,  1, 13, 12,  2,  6,  8, 11, 18, 16, 16, 16,
       18, 16, 21, 16,  3, 17, 19, 18,  7, 21, 12, 12, 18,  8, 15, 12, 10,
       13, 11,  1,  2, 11, 20, 11, 13,  3,  0,  5,  6,  7, 20,  1,  8,  8,
       12,  8,  9,  3, 10,  7, 10,  4,  4,  4, 19, 15,  3, 21, 20, 12,  2,
        4,  6,  3, 21,  3,  2,  9,  5,  5, 16,  2,  8,  3, 20, 11, 16,  6,
        7, 14,  3, 10,  0

In [37]:
# Accuracy
accuracy = accuracy_score(y_test, pred_index)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 39.60%


In [38]:
# save and load model
pickle.dump(gbm, open('lightgbm.pickle.dat', 'wb'))
loaded_model2 = pickle.load(open('lightgbm.pickle.dat', 'rb'))

In [40]:
# emotions
label = pd.read_csv("C:/Users/lyc07/Desktop/Columbia/S3/GR5243 Applied Data Science/prj3/data/train_set/label.csv")
emotion = label['emotion_cat'].unique()
emotions_pred = [emotion[idx] for idx in pred_index]

In [41]:
emotions_pred

['Fearfully surprised',
 'Angry',
 'Sadly angry',
 'Hatred',
 'Happy',
 'Sadly surprised',
 'Sadly surprised',
 'Disgustedly surprised',
 'Fearfully disgusted',
 'Fearfully disgusted',
 'Sadly fearful',
 'Happy',
 'Angrily surprised',
 'Angrily disgusted',
 'Happily disgusted',
 'Disgustedly surprised',
 'Sadly fearful',
 'Angrily surprised',
 'Disgustedly surprised',
 'Sadly angry',
 'Fearfully angry',
 'Disgustedly surprised',
 'Sadly disgusted',
 'Disgusted',
 'Angrily surprised',
 'Happily surprised',
 'Fearfully disgusted',
 'Happily disgusted',
 'Fearfully angry',
 'Sadly disgusted',
 'Angry',
 'Disgustedly surprised',
 'Sad',
 'Disgusted',
 'Angry',
 'Sad',
 'Disgustedly surprised',
 'Sadly surprised',
 'Angry',
 'Neutral',
 'Angrily disgusted',
 'Happily disgusted',
 'Surprised',
 'Sadly angry',
 'Hatred',
 'Happily surprised',
 'Sad',
 'Disgusted',
 'Sad',
 'Sadly fearful',
 'Sad',
 'Angrily surprised',
 'Sadly fearful',
 'Happily disgusted',
 'Sadly fearful',
 'Awed',
 'Disgu